In [19]:
conda install gensim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\gurde\Anaconda3\envs\deloitte

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boto3-1.9.66               |           py37_0         106 KB
    botocore-1.12.189          |             py_0         3.4 MB
    bz2file-0.98               |           py37_1          14 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2019.11.28         |           py37_0         154 KB
    docutils-0.16              |           py37_0         667 KB
    gensim-3.8.0               |   py37hf9181ef_0        18.4 MB
    jmespath-0.9.4             |             py_0          22 KB
    s3transfer-0.1.13          |           py37_0          79 KB
    smart_open-1.9.0           |             py_0          59 KB
    -------------------------------------

In [3]:
import re
import nltk
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [ ]:
import nltk
import re

In [4]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
data = pd.read_csv('datasets\\IMDB Dataset.csv')
news = pd.read_csv('datasets\\RedditNews.csv')
twitter = 

In [8]:
if(data.sentiment[0] not in [1, -1]):
    data.sentiment = np.where(data.sentiment == "positive", 4, 0)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,4
1,A wonderful little production. <br /><br />The...,4
2,I thought this was a wonderful way to spend ti...,4
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",4


In [5]:
s = SIA()
total_values, incorrect = 0, 0
for review, sentiment in tqdm(zip(data.review, data.sentiment)):
    pred = s.polarity_scores(review)
    if(pred['neg'] > pred['pos']):
        pred = -1
    else:
        pred = 1
    if(pred != sentiment):
        incorrect+=1
    total_values+=1
print("Accuracy Score:", (total_values-incorrect)/total_values * 100)


Accuracy Score: 69.22399999999999


In [13]:
train_x, test_x, train_y, test_y = tuple(map(lambda x: x.values, train_test_split(data.review, data.sentiment, test_size=0.2, random_state=42)))
print(len(train_x), len(train_y), len(test_x), len(test_y))

40000 40000 10000 10000


['That\'s what I kept asking myself during the many fights, screaming matches, swearing and general mayhem that permeate the 84 minutes. The comparisons also stand up when you think of the one-dimensional characters, who have so little depth that it is virtually impossible to care what happens to them. They are just badly written cyphers for the director to hang his multicultural beliefs on, a topic that has been done much better in other dramas both on TV and the cinema.<br /><br />I must confess, I\'m not really one for spotting bad performances during a film, but it must be said that Nichola Burley (as the heroine\'s slutty best friend) and Wasim Zakir (as the nasty, bullying brother) were absolutely terrible. I don\'t know what acting school they graduated from, but if I was them I\'d apply for a full refund post haste. Only Samina Awan in the lead role manages to impress in a cast of so-called British talent that we\'ll probably never hear from again. At least, that\'s the hope. N

In [11]:
clf = Pipeline([
('vectorizer', CountVectorizer(analyzer="word",
ngram_range=(1, 2),
tokenizer=word_tokenize,
preprocessor=clean_text)),
('classifier', LinearSVC(max_iter=2500, dual=False, random_state=42, verbose=2))])
clf.fit(train_x, train_y)
clf.score(test_x, test_y)

[LibLinear]

0.902

In [2]:
sentiment_dict = {0:"negative", 2:"neutral", 4:"positive"}

In [3]:
#start
imdb_data = pd.read_csv('datasets\\IMDB Dataset.csv')
twitter_data = pd.read_csv('datasets\\training.1600000.processed.noemoticon.csv', 
                           names=["target", "ids", "date", "flag", "user", "text"], 
                           encoding="ISO-8859-1")

In [4]:
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
# preparing data for training
if(imdb_data.sentiment[0] not in [4, 0]):
    imdb_data['sentiment'].replace({"positive":4, "negative":0}, inplace=True)
    #imdb_data.sentiment = np.where(imdb_data.sentiment == "positive", 1, -1)
print("Prepared IMDB Dataset\n", imdb_data.head())

if("flag" in twitter_data.keys()):
    twitter_data = twitter_data.drop(['user', 'flag', 'date', 'ids'], axis=1)
print("Prepared twitter Dataset\n", twitter_data.head())

Prepared IMDB Dataset
                                               review  sentiment
0  One of the other reviewers has mentioned that ...          4
1  A wonderful little production. <br /><br />The...          4
2  I thought this was a wonderful way to spend ti...          4
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          4
Prepared twitter Dataset
    target                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


In [4]:
#for cleaning text
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"}

def clean_text(text, remove_stopwords = False):
    text = text.lower()
    
    if(True):
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)

    text = re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text)
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)

    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [14]:
def sia(string, ret_decision=False):
    if(ret_decision == True):
        score = SIA().polarity_scores(string)['compound']
        if(score < -0.2):
            return 0
        elif(score > 0.2):
            return 4
        return 2
    return SIA().polarity_scores(string)

In [9]:
print("Original Text:", twitter_data.text[0])
print("Cleaned Text:", clean_text(twitter_data.text[0]))
print("SIA:", sia(clean_text(twitter_data.text[0])))
print("Original Sentiment:", twitter_data.target[0])

Original Text: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Cleaned Text:     awww that is a bummer you shoulda got david carr of third day to do it d
SIA: {'neg': 0.157, 'neu': 0.843, 'pos': 0.0, 'compound': -0.3818}
Original Sentiment: 0


In [10]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [134]:
total, correct = 0, 0
data = twitter_data[:5000]
for i in tqdm(range(int(len(data)))):
    if(sia(clean_text(twitter_data['text'][i]), True) == twitter_data['target'][i]):
        correct+=1
    total+=1
print("Accuracy:", correct/total*100)


Accuracy: 37.18


In [11]:
imdb_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,4
1,A wonderful little production. <br /><br />The...,4
2,I thought this was a wonderful way to spend ti...,4
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",4
...,...,...
49995,I thought this movie did a down right good job...,4
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [12]:
text_data = np.concatenate((np.asarray(twitter_data['text']), imdb_data['review']), axis=0)
sentiment_data = np.concatenate((np.asarray(twitter_data['target']), imdb_data['sentiment']), axis=0)
text_data.shape, sentiment_data.shape

((1650000,), (1650000,))

In [13]:
train_x, test_x, train_y, test_y = train_test_split(text_data, 
                                                    sentiment_data, 
                                                    test_size=0.10, 
                                                    shuffle=True,
                                                    random_state=42)
#train_x = train_x[:100000]
#test_x = test_x[:10000]
#train_y = train_y[:100000]
#test_y = test_y[:10000]
print(len(train_x), len(train_y), len(test_x), len(test_y))
#best iter = 2500 for 50000, 100000 samples

1485000 1485000 165000 165000


In [38]:
%%time
clf = Pipeline([
                ('vectorizer', CountVectorizer(analyzer="word",
                                               ngram_range=(1, 2),
                                               tokenizer=word_tokenize,
                                               #tokenizer=lambda text: mark_negation(word_tokenize(text)),
                                               preprocessor=clean_text,
                                               decode_error={'strict'})),#encoding="ISO-8859-1"
                ('classifier', LinearSVC(max_iter=1000, dual=False, random_state=42))])
clf['vectorizer'].fit(train_x, train_y)
#clf.score(test_x, test_y)

C:\Users\gurde\Anaconda3\envs\deloitte\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Wall time: 4min 16s


CountVectorizer(analyzer='word', binary=False, decode_error={'strict'},
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2),
                preprocessor=<function clean_text at 0x00000262C8B718B8>,
                stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x00000262C61A8DC8>,
                vocabulary=None)

In [63]:
clf['vectorizer'].inverse_transform(["Iran Bombed US"])

[array(['0'], dtype='<U145')]

In [ ]:
vectorizer = clf['vectorizer']
n_clf = Pipeline([('vectorizer', vectorizer),
                  ('classifier', LinearSVC(max_iter=1000, dual=False, random_state=42))])
n_clf.fit(train_x, train_y)

In [192]:
acc = []

# Iterate along a logarithmically spaced ranged
for i in tqdm(range(2500, 4000, 500)):
    # Print out the number of iterations to use for the current loop
    print('Training model with iterations: ', i)
    clf = Pipeline([
                ('vectorizer', CountVectorizer(analyzer="word",
                                               ngram_range=(1, 2),
                                               tokenizer=word_tokenize,
                                               #tokenizer=lambda text: mark_negation(word_tokenize(text)),
                                               preprocessor=clean_text,
                                               decode_error={'strict'})),#encoding="ISO-8859-1"
                ('classifier', LinearSVC(max_iter=i, dual=False, random_state=42))])
    # Fit the algorithm to the data
    clf.fit(train_x, train_y)
    # Append the current accuracy score to the template list
    acc.append(accuracy_score(test_y, clf.predict(test_x)) * 100)
print(acc)
# Convert the accuracy list to a series
acc = pd.Series(acc, index = list(range(1000, 3000, 250)))
# Set the plot size
plt.figure(figsize = (15,10))
# Set the plot title
title = 'Graph to show the accuracy of the SVC model as number of iterations increases\nfinal accuracy: ' + str(acc.iloc[-1])
plt.title(title)
# Set the xlabel and ylabel
plt.xlabel('Number of iterations')
plt.ylabel('Accuracy score')
# Plot the graph
acc.plot.line()
plt.show()

Training model with iterations:  2500


C:\Users\gurde\Anaconda3\envs\deloitte\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training model with iterations:  3000
Training model with iterations:  3500

[77.98, 77.94, 77.91]


ValueError: Length of passed values is 3, index implies 8.

In [31]:
clf['vectorizer'].__dir__()

['input',
 'encoding',
 'decode_error',
 'strip_accents',
 'preprocessor',
 'tokenizer',
 'analyzer',
 'lowercase',
 'token_pattern',
 'stop_words',
 'max_df',
 'min_df',
 'max_features',
 'ngram_range',
 'vocabulary',
 'binary',
 'dtype',
 'fixed_vocabulary_',
 '_stop_words_id',
 'stop_words_',
 'vocabulary_',
 '__module__',
 '__doc__',
 '__init__',
 '_sort_features',
 '_limit_features',
 '_count_vocab',
 'fit',
 'fit_transform',
 'transform',
 'inverse_transform',
 'get_feature_names',
 '_more_tags',
 '_white_spaces',
 'decode',
 '_word_ngrams',
 '_char_ngrams',
 '_char_wb_ngrams',
 'build_preprocessor',
 'build_tokenizer',
 'get_stop_words',
 '_check_stop_words_consistency',
 '_validate_custom_analyzer',
 'build_analyzer',
 '_validate_vocabulary',
 '_check_vocabulary',
 '_validate_params',
 '_warn_for_unused_params',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__

In [18]:
clf.score(np.asarray(imdb_data.review[:]), np.asarray(imdb_data.sentiment[:]))

0.77774

In [65]:
from sklearn.model_selection import validation_curve, learning_curve
import matplotlib.pyplot as plt

In [195]:
test_scores = np.empty(test_y.shape, dtype='int32')
train_scores = np.empty(train_y.shape, dtype='int32')
for i, (text, target) in tqdm(enumerate(zip(test_x, test_y))):
    test_scores[i] = clf.predict([text])
for i, (text, target) in tqdm(enumerate(zip(train_x, train_y))):
    train_scores[i] = clf.predict([text])
test_scores, train_scores

KeyboardInterrupt: 

In [16]:
data = np.asarray(pd.read_csv('datasets\\RedditNews.csv').News)

In [25]:
#total, correct = 0, 0
for news in tqdm(data[11140:11150]):
    print("News Headline:", news)
    print("SIA:", sia(news))
    print(l_clf.predict([news])[0])
    #if(sia(clean_text(news), True) == clf.predict([news])[0]):
    #    if(correct < 10):
    #        print("News Headline:", news)
    #        print("SIA:", sia(news))
    #        print(sentiment_dict[clf.predict([news])[0]])
    #    elif(correct >= 10):
    #        break
    #    correct+=1
    #total+=1
        
#print("Accuracy:", (correct/total)*100)

News Headline: After deploying 3,000 troops and spending hundreds of millions of dollars, only 28 Ebola patients have been treated at the 11 centers built by the American military in Liberia
SIA: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
4
News Headline: 46 Ukrainian banks declare insolvency:Dozens of Ukrainian banks have reportedly gone bankrupt over the course of one year.
SIA: {'neg': 0.175, 'neu': 0.825, 'pos': 0.0, 'compound': -0.5574}
0
News Headline: More than 100 hand grenades and around a dozen automatic weapons on their way to Stockholm were confiscated on Friday when Bosnian police swooped down in on a weapons ring that had been on law enforcements radar since last year.
SIA: {'neg': 0.131, 'neu': 0.791, 'pos': 0.078, 'compound': -0.3262}
0
News Headline: German foreign minister rejects calls to invite Putin to G7 talks
SIA: {'neg': 0.232, 'neu': 0.652, 'pos': 0.116, 'compound': -0.3818}
4
News Headline: South African Author ZP Dala Reportedly Taken to Mental Ins

In [18]:
%%time
import joblib
joblib.dump(clf, "test.joblib", compress=1)

['imdb_clf.joblib']

In [5]:
%%time
import joblib
l_clf = joblib.load("test_model1.joblib")

Wall time: 8.11 s


In [13]:
line = "After deploying 3,000 troops and spending hundreds of millions of dollars, only 28 Ebola patients have been treated at the 11 centers built by the American military in Liberia"
for i in line.split(" "):
    print(i, l_clf.predict([i]))

After [4]
deploying [4]
3,000 [0]
troops [4]
and [4]
spending [4]
hundreds [4]
of [4]
millions [4]
of [4]
dollars, [4]
only [4]
28 [4]
Ebola [4]
patients [0]
have [4]
been [4]
treated [4]
at [4]
the [4]
11 [4]
centers [4]
built [4]
by [4]
the [4]
American [4]
military [4]
in [4]
Liberia [4]
